In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

import sys

print(f'PID number is {os.getpid()}')

sys.path.append(r'C:\Users\cosmosyw\Documents\Softwares')
import ImageAnalysis3 as ia
from ImageAnalysis3.classes import _allowed_kwds
from ImageAnalysis3 import *
import time
import h5py
import ast

PID number is 35188


In [2]:
# Required plotting setting
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
plt.rc('font', family='serif')
plt.rc('font', serif='Arial')
_font_size = 7.5

from ImageAnalysis3.figure_tools import _single_col_width, _font_size, _ticklabel_size,_ticklabel_width

In [3]:
postanalysis_folder = r'E:\DNA_analysis\Postanalysis_MeCP2'
print("PostAnalysis Folder:", postanalysis_folder)

output_folder = r'AB_ratio_subclass'

figure_folder = output_folder
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

PostAnalysis Folder: E:\DNA_analysis\Postanalysis_MeCP2


# Load codebook

In [4]:
codebook_folder = r'\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis'
df_refgen_savename = os.path.join(codebook_folder,'refgen_df_for_jie.csv')
df_refgen=pd.read_csv(df_refgen_savename,index_col=0)
df_refgen['chr'] = df_refgen['chr'].apply(lambda x: 'chr'+x)
df_refgen.head()

,name,id,chr,chr_order,library,dtype,loci_name,start,end,hyb,delta,region_id
0,1:3742742-3759944,1,chr1,0,CTP11,combo,chr1_3742742_3759944,3742742,3759944,0,0,1
1,1:6245958-6258969,2,chr1,1,CTP11,combo,chr1_6245958_6258969,6245958,6258969,1,2503216,2
2,1:8740008-8759916,3,chr1,2,CTP11,combo,chr1_8740008_8759916,8740008,8759916,2,4997266,3
3,1:9627926-9637875,1,chr1,3,CTP13,combo,chr1_9627926_9637875,9627926,9637875,3,5885184,4
4,1:9799472-9811359,2,chr1,4,CTP13,combo,chr1_9799472_9811359,9799472,9811359,4,6056730,5


In [5]:
chrom_number = []
for i, row in df_refgen.iterrows():
    if ('X' in row.chr):
        chrom_number.append(20)
    elif ('Y' in row.chr):
        chrom_number.append(21)
    else:
        chrom_number.append(int(row['chr'].split('chr')[-1]))
df_refgen['chrom_number'] = chrom_number
df_refgen.sort_values(['chrom_number','hyb'], inplace=True, ignore_index=True)
df_refgen.head()

,name,id,chr,chr_order,library,dtype,loci_name,start,end,hyb,delta,region_id,chrom_number
0,1:3742742-3759944,1,chr1,0,CTP11,combo,chr1_3742742_3759944,3742742,3759944,0,0,1,1
1,1:6245958-6258969,2,chr1,1,CTP11,combo,chr1_6245958_6258969,6245958,6258969,1,2503216,2,1
2,1:8740008-8759916,3,chr1,2,CTP11,combo,chr1_8740008_8759916,8740008,8759916,2,4997266,3,1
3,1:9627926-9637875,1,chr1,3,CTP13,combo,chr1_9627926_9637875,9627926,9637875,3,5885184,4,1
4,1:9799472-9811359,2,chr1,4,CTP13,combo,chr1_9799472_9811359,9799472,9811359,4,6056730,5,1


In [6]:
from AnalysisTool_Chromatin_SL import loci_1d_features

codebook_df = pd.read_csv(r'\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis_20230201\merged_codebook_w_celltype_w_rank.csv')

# sort df temporailiy so matrix can be sliced by df order directly
codebook_df = loci_1d_features.sort_loci_df_by_chr_order (codebook_df)

codebook_df.head()

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_460,NDB_563,NDB_592,NDB_368,NDB_436,NDB_629,NDB_604,cell_type,enhancer_rank,enhancer_name
0,1:3742742-3759944,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,['None'],['None'],['None']
1,1:6245958-6258969,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,['None'],['None'],['None']
2,1:8740008-8759916,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,['None'],['None'],['None']
1016,1:9627926-9637875,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,['L6_CT'],[568],1:9610471-9646458
1017,1:9799472-9811359,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,['L2_3_IT'],[991],1:9759317-9835402


In [7]:
sort_by_region = False
from ImageAnalysis3.structure_tools import distance

chr_2_indices, chr_2_orders = distance.Generate_PlotOrder(codebook_df, codebook_df, sort_by_region=sort_by_region)

# Load AB assignment

In [8]:
# load AB assignment from wild-type MOp dataset
AB_summary_fname =  r"C:\Users\cosmosyw\Documents\Research\Mecp2_v2\results\AB_assignment_shared_ABth_zero.csv"
AB_summary_df = pd.read_csv(AB_summary_fname, index_col=0)

In [9]:
AB_summary_df

,Astro,Endo,GABA,Gluta,Micro,Oligo
loci_name,,,,,,
chr1_3742742_3759944,B,B,B,B,B,B
chr1_6245958_6258969,A,B,B,B,B,B
chr1_8740008_8759916,B,B,B,B,B,B
chr1_9627926_9637875,A,B,B,B,A,B
chr1_9799472_9811359,A,B,B,B,A,A
...,...,...,...,...,...,...
chrX_163750534_163758706,A,A,A,A,A,A
chrX_166247682_166259932,A,A,A,A,A,A
chrX_167157164_167167452,A,A,A,A,A,A


In [10]:
celltype_chr_2_AB_dict = {}
for _group in AB_summary_df.columns:
    
    celltype_AB_dict = {}
    celltype_AB_dict['A'] = np.where(np.array(AB_summary_df[_group].tolist())=='A')[0]
    celltype_AB_dict['B'] = np.where(np.array(AB_summary_df[_group].tolist())=='B')[0]
    # by chr
    celltype_chr_2_AB = {}
    for _chr in chr_2_indices:
        _chr_AB = {
            'A':np.array([list(chr_2_indices[_chr]).index(_ind) 
                          for _ind in np.intersect1d(chr_2_indices[_chr], celltype_AB_dict['A'])], dtype=np.int32),
            'B':np.array([list(chr_2_indices[_chr]).index(_ind) 
                          for _ind in np.intersect1d(chr_2_indices[_chr], celltype_AB_dict['B'])], dtype=np.int32),
        }
        celltype_chr_2_AB[_chr] = _chr_AB
    
    # append dict
    celltype_chr_2_AB_dict[_group+'_WT'] = celltype_chr_2_AB
    celltype_chr_2_AB_dict[_group+'_KO'] = celltype_chr_2_AB

# Calculate AB score and density ratio

## subclass level using majorType AB

In [11]:
#postanalysis_folder = r'\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis_0920'
class_2_chr2Zxys_filename = os.path.join(postanalysis_folder, 'subclass_2_chr2Zxys.pkl')
class_2_chr2ZxysList = pickle.load(open(class_2_chr2Zxys_filename, 'rb'))

In [12]:
from tqdm import tqdm
from ImageAnalysis3.compartment_tools.density import BatchCompartmentDensities

In [13]:
class_2_subclass_names = {
    'Gluta':['L6b', 'L6 CT', 'L6 IT', 'L5 IT', 'L4/5 IT', 'L2/3 IT', 'L5/6 NP', 'L5 ET', ],
    'GABA':['Sncg', 'Sst', 'Vip', 'Pvalb', 'Lamp5', ],
    'Astro':['Astro', ],
    'Endo':['Endo-PVM', 'SMC-VLMC', 'Peri'],
    'Micro':['Micro', ],
    'Oligo':['Oligo', 'OPC', ],
}
subclass_2_class = {}
class_2_subclass = {}
for _cls in class_2_subclass_names:
    class_2_subclass[_cls + '_' + 'WT'] = []
    class_2_subclass[_cls + '_' + 'KO'] = []
    for _subcls in class_2_subclass_names[_cls]:
        subclass_2_class[_subcls+'_'+'WT'] = _cls + '_' + 'WT'
        subclass_2_class[_subcls+'_'+'KO'] = _cls + '_' + 'KO'
        class_2_subclass[_cls + '_' + 'WT'].append(_subcls+'_'+'WT')
        class_2_subclass[_cls + '_' + 'KO'].append(_subcls+'_'+'KO')

### AB score

In [14]:
gaussian_radius = 0.5 # um
normalize_by_reg_num = False
num_threads=24

class_2_transScoreDicts = {}
class_2_cisScoreDicts = {}
for _sel_class in subclass_2_class.keys():
    majorType_name = subclass_2_class[_sel_class]
    celltype_chr_2_AB = celltype_chr_2_AB_dict[majorType_name]
    
    print(_sel_class, end =' ')
    _class_start = time.time()
    class_2_transScoreDicts[_sel_class] = BatchCompartmentDensities(
        class_2_chr2ZxysList[_sel_class], celltype_chr_2_AB, 
        gaussian_radius, num_threads=num_threads,
        normalize_by_reg_num=normalize_by_reg_num,)
   
    print(f"in {time.time()-_class_start:.2f}s. ")

L6b_WT in 11.64s. 
L6b_KO in 13.13s. 
L6 CT_WT in 49.43s. 
L6 CT_KO in 72.00s. 
L6 IT_WT in 26.31s. 
L6 IT_KO in 38.47s. 
L5 IT_WT in 26.17s. 
L5 IT_KO in 36.55s. 
L4/5 IT_WT in 17.46s. 
L4/5 IT_KO in 27.15s. 
L2/3 IT_WT in 26.84s. 
L2/3 IT_KO in 36.84s. 
L5/6 NP_WT in 11.75s. 
L5/6 NP_KO in 14.33s. 
L5 ET_WT in 12.95s. 
L5 ET_KO in 16.50s. 
Sncg_WT in 6.94s. 
Sncg_KO in 7.40s. 
Sst_WT in 13.07s. 
Sst_KO in 13.97s. 
Vip_WT in 8.94s. 
Vip_KO in 9.06s. 
Pvalb_WT in 14.86s. 
Pvalb_KO in 17.40s. 
Lamp5_WT in 8.67s. 
Lamp5_KO in 8.57s. 
Astro_WT in 15.54s. 
Astro_KO in 21.62s. 
Endo-PVM_WT in 18.68s. 
Endo-PVM_KO in 18.00s. 
SMC-VLMC_WT in 7.58s. 
SMC-VLMC_KO in 7.86s. 
Peri_WT in 10.86s. 
Peri_KO in 12.80s. 
Micro_WT in 9.92s. 
Micro_KO in 10.65s. 
Oligo_WT in 39.28s. 
Oligo_KO in 55.11s. 
OPC_WT in 10.81s. 
OPC_KO in 12.28s. 


In [15]:
# save result
transABscores_ensemble_filename = os.path.join(output_folder, f'transABscores_thresh0_r{gaussian_radius}_bysubclass.pkl')
if not os.path.exists(transABscores_ensemble_filename):
    print(transABscores_ensemble_filename)
    pickle.dump(class_2_transScoreDicts, open(transABscores_ensemble_filename, 'wb'))

AB_ratio_subclass\transABscores_thresh0_r0.5_bysubclass.pkl


### AB ratio

In [16]:
# convert into dict
class_2_transABRatioDicts = {}
# loop
for _class, _scoreDicts in class_2_transScoreDicts.items():
    print(_class, len(_scoreDicts))
    
    _ABratioDicts = [] 
    for _scoreDict in _scoreDicts:
        _chr_2_abratios = {}
        for _chr, _chr_AB_dict in _scoreDict.items():
            _AB_ratios = np.log(_chr_AB_dict['A']) - np.log(_chr_AB_dict['B'])
            _chr_2_abratios[_chr] = _AB_ratios
        # append
        _ABratioDicts.append(_chr_2_abratios)
    # assign
    class_2_transABRatioDicts[_class] = _ABratioDicts

L6b_WT 379
L6b_KO 484
L6 CT_WT 2944
L6 CT_KO 4093
L6 IT_WT 1363
L6 IT_KO 1998
L5 IT_WT 1388
L5 IT_KO 2067
L4/5 IT_WT 868
L4/5 IT_KO 1575
L2/3 IT_WT 1606
L2/3 IT_KO 2665
L5/6 NP_WT 381
L5/6 NP_KO 600
L5 ET_WT 560
L5 ET_KO 827
Sncg_WT 117
Sncg_KO 132


C:\Users\cosmosyw\AppData\Local\Temp\ipykernel_35188\2276297677.py:11: RuntimeWarning: divide by zero encountered in log
  _AB_ratios = np.log(_chr_AB_dict['A']) - np.log(_chr_AB_dict['B'])
C:\Users\cosmosyw\AppData\Local\Temp\ipykernel_35188\2276297677.py:11: RuntimeWarning: invalid value encountered in subtract
  _AB_ratios = np.log(_chr_AB_dict['A']) - np.log(_chr_AB_dict['B'])


Sst_WT 569
Sst_KO 628
Vip_WT 285
Vip_KO 307
Pvalb_WT 695
Pvalb_KO 923
Lamp5_WT 204
Lamp5_KO 238
Astro_WT 1476
Astro_KO 2150
Endo-PVM_WT 1537
Endo-PVM_KO 1508
SMC-VLMC_WT 181
SMC-VLMC_KO 285
Peri_WT 413
Peri_KO 652
Micro_WT 394
Micro_KO 405
Oligo_WT 2503
Oligo_KO 3542
OPC_WT 348
OPC_KO 455


In [17]:
transABratio_ensemble_filename = os.path.join(output_folder, f'transABRatio_thresh0_r{gaussian_radius}_bysubclass.pkl')
if not os.path.exists(transABratio_ensemble_filename):
    print(transABratio_ensemble_filename)
    pickle.dump(class_2_transABRatioDicts, open(transABratio_ensemble_filename, 'wb'))

AB_ratio_subclass\transABRatio_thresh0_r0.5_bysubclass.pkl
